In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset

dataset_folder = '/kaggle/input/noisy-drone-rf-signal-classification/'

# load data
dataset_dict = torch.load(dataset_folder + 'dataset.pt')
dataset_dict.keys()

x_iq = dataset_dict['x_iq']
x_spec = dataset_dict['x_spec']
y = dataset_dict['y']
snrs = dataset_dict['snr']
duty_cycle = dataset_dict['duty_cycle']

In [4]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.fft import fft

# Function to extract features from spectrograms
def extract_spectrogram_features(spectrogram):
    mean_val = np.mean(spectrogram)
    median_val = np.median(spectrogram)
    std_val = np.std(spectrogram)
    skewness = skew(spectrogram.flatten())
    kurt = kurtosis(spectrogram.flatten())
    return [mean_val, median_val, std_val, skewness, kurt]

def extract_iq_features(iq_data):
    # Assuming iq_data is a 2D array with shape (2, N)
    iq_features = []

    for channel in iq_data:
        mean_val = np.mean(channel)
        median_val = np.median(channel)
        std_val = np.std(channel)
        skewness = skew(channel)
        kurt = kurtosis(channel)

        # Frequency domain features
        freq_domain = fft(channel)
        dominant_freq = np.argmax(np.abs(freq_domain))
        freq_variance = np.var(np.abs(freq_domain))

        iq_features.extend([mean_val, median_val, std_val, skewness, kurt, dominant_freq, freq_variance])

    return iq_features

# Function to create a DataFrame with extracted features
def create_feature_dataframe(x_iq, x_spec, y, snrs, duty_cycle):
    features = []
    for iq, spec, label, snr, duty in zip(x_iq, x_spec, y, snrs, duty_cycle):
        iq = iq.numpy()
        spec = spec.numpy()
        spectrogram_features = extract_spectrogram_features(spec)
        iq_features = extract_iq_features(iq)
        features.append(spectrogram_features + iq_features + [snr.item(), duty.item(), label.item()])

    columns = ['spec_mean', 'spec_median', 'spec_std', 'spec_skew', 'spec_kurt',
               'iq_mean_ch1', 'iq_median_ch1', 'iq_std_ch1', 'iq_skew_ch1', 'iq_kurt_ch1', 'iq_dom_freq_ch1', 'iq_freq_var_ch1',
               'iq_mean_ch2', 'iq_median_ch2', 'iq_std_ch2', 'iq_skew_ch2', 'iq_kurt_ch2', 'iq_dom_freq_ch2', 'iq_freq_var_ch2',
               'snr', 'duty_cycle', 'label']
    df = pd.DataFrame(features, columns=columns)
    return df

# Extracting features and creating DataFrame
feature_df = create_feature_dataframe(x_iq, x_spec, y, snrs, duty_cycle)

# Save to CSV
feature_df.to_csv('feature_data.csv', index=False)

In [19]:
from IPython.display import FileLink
FileLink(r'feature_data.csv')

/kaggle/working/feature_data.csv